In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import furniture_bench
import gym
import torch

from src.models.vision import get_encoder

## Test normal env

In [ ]:
env = gym.make(
    "FurnitureSim-v0",
    furniture="one_leg",  # Specifies the type of furniture [lamp | square_table | desk | drawer | cabinet | round_table | stool | chair | one_leg].
    num_envs=16,  # Number of parallel environments.
    resize_img=True,  # If true, images are resized to 224 x 224.
    concat_robot_state=True,  # If true, robot state is concatenated to the observation.
    headless=True,  # If true, simulation runs without GUI.
    compute_device_id=0,
    graphics_device_id=0,
    init_assembled=False,  # If true, the environment is initialized with assembled furniture.
    np_step_out=False,  # If true, env.step() returns Numpy arrays.
    channel_first=False,  # If true, images are returned in channel first format.
    randomness="low",  # Level of randomness in the environment [low | med | high].
    high_random_idx=-1,  # Index of the high randomness level (range: [0-2]). Default -1 will randomly select the index within the range.
    save_camera_input=False,  # If true, the initial camera inputs are saved.
    record=False,  # If true, videos of the wrist and front cameras' RGB inputs are recorded.
    max_env_steps=3000,  # Maximum number of steps per episode.
    act_rot_repr="quat",  # Representation of rotation for action space. Options are 'quat' and 'axis'.
)

In [3]:
# Sample action
action = env.action_space.sample()
print("Action shape: ", action.shape)

Action shape:  (16, 8)


In [4]:
env.reset()

# Take a step
obs, reward, done, info = env.step(action)

In [6]:
obs["color_image2"].device

device(type='cuda', index=0)

In [7]:
obs["robot_state"].shape

torch.Size([8, 14])

## Test Feature Env

In [3]:
encoder = get_encoder("dinov2-base")

env = gym.make(
    "FurnitureSimFeature-v0",
    furniture="one_leg",  # Specifies the type of furniture [lamp | square_table | desk | drawer | cabinet | round_table | stool | chair | one_leg].
    num_envs=4,  # Number of parallel environments.
    headless=True,  # If true, simulation runs without GUI.
    encoder=encoder,
    compute_device_id=0,
    graphics_device_id=0,
    init_assembled=False,  # If true, the environment is initialized with assembled furniture.
    randomness="low",  # Level of randomness in the environment [low | med | high].
    high_random_idx=-1,  # Index of the high randomness level (range: [0-2]). Default -1 will randomly select the index within the range.
    save_camera_input=False,  # If true, the initial camera inputs are saved.
    record=False,  # If true, videos of the wrist and front cameras' RGB inputs are recorded.
    max_env_steps=3000,  # Maximum number of steps per episode.
    act_rot_repr="quat",  # Representation of rotation for action space. Options are 'quat' and 'axis'.
)

PyTorch version 2.0.1+cu117
Device count 2
/home/larsankile/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/larsankile/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
Emitting ninja build file /home/larsankile/.cache/torch_extensions/py38_cu117/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module gymtorch...


ninja: no work to do.


AttributeError: module 'furniture_bench.envs.policy_envs.furniture_sim_image_feature' has no attribute 'FurnitureSimFeature'

In [8]:
obs = env.reset()

> /home/larsankile/furniture-bench/furniture_bench/envs/policy_envs/furniture_sim_image_feature.py(126)_get_observation()
     92             ret["color_image2"] = image2_raw
     93 
     94         return ret



In [6]:
encoder(obs["color_image2"]).shape

NameError: name 'obs' is not defined

In [4]:
encoder = get_encoder("dinov2-base")

In [12]:
encoder(torch.zeros(16, 2, 224, 224, 3).reshape(-1, 224, 224, 3)).reshape(
    16, 2, -1
).shape

torch.Size([16, 2, 768])